In [88]:
import os
import pandas as pd
import json
import spacy
import textacy
import re
from textacy.preprocessing import remove_punctuation, replace_emojis, replace_urls
import textacy.vsm 
from textacy.vsm import Vectorizer
import textacy.tm
from flair.data import Sentence
from flair.models import SequenceTagger
import pandas as pd
import os
from textblob import TextBlob
import multiprocessing as mp
import ast
from twitteremotion.emotion_predictor import EmotionPredictor
import preprocessor as p
import json
p.set_options(p.OPT.URL, p.OPT.EMOJI)
os.environ['KERAS_BACKEND'] = 'theano'

In [93]:
path = "rawDAta/twitter_MekongRiver_disease_YHNov12.csv"
outPath = "processedData/"

In [39]:
df = pd.read_csv(path)

In [41]:
other_stops = ["t","co","s", "t co","g",'s   t co',"https","htt"]

In [42]:
def getEntities(tweet):
  blob = TextBlob(tweet)
  result = []
  for sent in blob.sentences:
    sent = str(sent)
    sentence = Sentence(sent)
    tagger.predict(sentence)
    entities = sentence.to_dict(tag_type='ner')
    for ent in entities["entities"]:
      result.append([ent["text"],ent["labels"]])
  return result

In [76]:
def clean_text(text):
  text = p.clean(text)
  text = remove_punctuation(text)
  text = replace_emojis(text)
  text = replace_urls(text)
  text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text)
  text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''',"",text)
  text = re.sub(r'[\d-]',"",text)
  return text

In [77]:
df["clean_text"] = df["text"].apply(clean_text)

In [46]:
emot_model = EmotionPredictor(classification='ekman', setting='mc')

In [48]:
predictions = emot_model.predict_probabilities(df["text"])

In [49]:
for emotion in ["Anger",	"Disgust",	'Fear',	"Joy",	"Sadness",	"Surprise"]:
  df[emotion] = predictions[emotion]

Entity

In [9]:
tagger = SequenceTagger.load('ner')

2020-12-14 10:17:46,251 https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt not found in cache, downloading to /var/folders/vq/gwn5ph656jxbd2spxww2bv_w0000gn/T/tmpwkf17erd


100%|██████████| 432197603/432197603 [01:52<00:00, 3838180.85B/s] 

2020-12-14 10:19:39,268 copying /var/folders/vq/gwn5ph656jxbd2spxww2bv_w0000gn/T/tmpwkf17erd to cache at /Users/darts/.flair/models/en-ner-conll03-v0.4.pt


2020-12-14 10:19:39,536 removing temp file /var/folders/vq/gwn5ph656jxbd2spxww2bv_w0000gn/T/tmpwkf17erd
2020-12-14 10:19:39,565 loading file /Users/darts/.flair/models/en-ner-conll03-v0.4.pt


In [51]:
ents = [getEntities(text) for text in df["text"]]

In [52]:
df["ents"] = ents

In [53]:
AllLocEnts = []
for entList in df["ents"]:
  locEnts = []
  for ent in entList:
    entText = ent[0]
    entLabel = ent[1][0].to_dict()
    if entLabel["value"] == "LOC":
      locEnts.append(entText)
  AllLocEnts.append(locEnts)

Topic

In [54]:
df["locEnts"] = AllLocEnts

In [80]:
df["entity"] = df["locEnts"].apply(lambda x: x[0] if len(x)>0 else "")

In [81]:
num = 3
en = textacy.load_spacy_lang("enn") #, disable=("parser",)

In [82]:
corpus = textacy.corpus.Corpus(lang=en, data=[i for i in df["clean_text"]]) #data=texts

In [83]:
tokenized_docs = (doc._.to_terms_list(entities=False, as_strings=True, normalize = "lower") for doc in corpus) #  
tokenized_docs = ([i for i in token_list if i not in other_stops and len(i)>2] for token_list in tokenized_docs)
# do tf-idf
vectorizer = Vectorizer(norm="l2", apply_idf=True, max_df=0.95) # , vocabulary_terms = s, max_df=0.95, min_df=5, norm="l2", apply_idf=True, 
doc_term_matrix = vectorizer.fit_transform(tokenized_docs)

In [84]:
model = textacy.tm.TopicModel("nmf", n_topics=20)
model.fit(doc_term_matrix)
doc_topic_matrix = model.transform(doc_term_matrix)
doc_topic_matrix.shape
topicTermsDict = {}
for topic_idx, top_terms in model.top_topic_terms(vectorizer.id_to_term, top_n=20):
    topicTermsDict[topic_idx] = list(top_terms)

In [85]:
doc_topics = list(model.top_doc_topics(doc_topic_matrix=doc_topic_matrix,docs=-1,top_n=1))

In [86]:
df["topic_id"] = [i[1][0] for i in doc_topics]

In [87]:
topicTermsDict

{0: ['covid',
  'thairath news covid',
  'thairath',
  'news covid',
  'news',
  'justice services',
  'informational signboards',
  'informational signboards vandalised',
  'innovations',
  'job',
  'jobs',
  'jobs during coronavirus',
  'justice',
  'khng',
  'justice services need',
  'infections',
  'khng c',
  'know',
  'kong',
  'kong philharmonic'],
 1: ['students',
  'test',
  'let covid',
  'class',
  'undergrad',
  'undergrad students',
  'paperless test',
  'paperless',
  'innovations',
  'education',
  'giving',
  'giving paperless',
  'stop',
  'giving paperless test',
  'stop education',
  'let',
  'hello sofia ra',
  'jobs during coronavirus',
  'hanoi',
  'jobs'],
 2: ['today s',
  'covid report',
  'report',
  'thailand shows',
  'deaths',
  'shows',
  'cases from state',
  'zero',
  'zero deaths',
  'new',
  'coronavirus cases',
  'state',
  'quarantine',
  'cases',
  'amp',
  'quarantine and zero',
  'jobs during coronavirus',
  'kong philharmonic',
  'innovations',


In [94]:
df.to_csv(outPath+"dataWithEmotionEntityTopic.csv")
with open(outPath+"topicTermDict.json","w") as jsonFile:
    jsonFile.write(json.dumps(topicTermsDict))